In [ ]:
import os
import re
from get_input import get_input

os.chdir('/Users/soup/Documents/projects/advent-of-code')

In [ ]:
input = get_input(2015, 19)

query_str  = ""
instructions = {}
replacements = set()

def init_data(str_input):
    for line in str_input.splitlines()[:-2]:
        original, _, replaced = line.split(" ")
        
        if original not in instructions:
            instructions[original] = []
            instructions[original].append(replaced)
        else: instructions[original].append(replaced)
        
        replacements.add(replaced)

    return str_input.splitlines()[-1:][0]
    
query_str = init_data(input)

In [ ]:
combined_pattern = '|'.join(map(re.escape, instructions.keys()))

terminals = set()

for word in replacements:
    terminals.add(re.sub(combined_pattern, "", word))

print(terminals)

pattern = 'Rn(.*?)Ar'

matches = re.findall(pattern, query_str, re.DOTALL)

In [ ]:
print(matches)
print(query_str)

In [ ]:
def replacer(instructions:tuple, string=query_str) -> list:
    created_molecules = []
    
    inds = [match.start() for match in re.finditer(instructions[0], string)]
    
    for element in inds:
        temp_s = string[:element] + instructions[1] + string[element+len(instructions[0]):]
        created_molecules.append(temp_s)
    
    return created_molecules

In [ ]:
def simplify(start):
    step = set([start])
    while True:
        inner = set()
        for item in step:
            for key, values in instructions.items():
                for element in values:
                    mols = replacer((element, key), item)
                    for mol in mols:
                        inner.add(mol)
        store = set()
        if len(inner) == 0:
            store.add(item)
        for key in instructions:
            if "Rn"+key+"Ar" in inner or "Rn"+key+"Ar" in step or 'e' in step:
                store.add("Rn"+key+"Ar")
        if len(store) >= 1:
            return list(store)[0]
        
        if step == inner:
            print("Inf loop")
            break
        
        step = inner
    
    return -1
                    

In [ ]:
repl = []
for word in matches:
    repl.append(simplify(word))

no_illegals = re.sub('Rn.*?Ar', "Rn" + repl.pop() + "Ar", query_str)
print(no_illegals)

modified_string = no_illegals

# Continue until there are no more 'Ar' patterns that can be replaced
while True:
    inds = [match.start() for match in re.finditer('Ar', modified_string)]
    made_replacement = False

    for ind in inds:
        # Start from the position just before 'Ar' and move backwards to find a valid replacement
        for i in range(ind, 0, -1):
            snippet = modified_string[ind-i:ind+2]
            if snippet in replacements:
                # Find the key in 'instructions' corresponding to the snippet
                replacement_key = next((key for key, values in instructions.items() if snippet in values), None)
                if replacement_key:
                    # Perform the replacement and update the modified string
                    modified_string = re.sub(re.escape(snippet), replacement_key, modified_string, count=1)
                    made_replacement = True
                    break  # Break after the first replacement

    # If no replacement was made in this iteration, break the loop
    if not made_replacement:
        break

# 'modified_string' now contains all the changes from the loop iterations
no_illegals = modified_string

print(no_illegals)